In [19]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [20]:
import sys
import importlib
sys.path.append('..')
import preprocessing.pipeline
importlib.reload(preprocessing.pipeline)
from preprocessing.pipeline import get_preprocessor

In [21]:
train = pd.read_csv("../data/train.csv")
X_test = pd.read_csv("../data/test.csv")

X_train = train.drop("SalePrice", axis=1)
y_train = train["SalePrice"]

In [22]:
# === Build preprocessing + modeling pipeline ===
model_pipeline = Pipeline([
  ("preprocessor", get_preprocessor()),
  ("model", RandomForestRegressor(n_estimators=100, random_state=42))
])

# === Fit the model ===
model_pipeline.fit(X_train, np.log1p(y_train)) # log(1 + y) to handle zeros

# === Predict on training data ===
y_pred_log = model_pipeline.predict(X_train)

# === Inverse transform the prediction ===
y_pred = np.expm1(y_pred_log)  # inverse of log1p

# === Evaluate ===
rmse = np.sqrt(mean_squared_error(y_train, y_pred))
r2 = r2_score(y_train, y_pred)

print(f"Train RMSE: {rmse:.2f}")
print(f"Train R² Score: {r2:.4f}")

Train RMSE: 10748.37
Train R² Score: 0.9817


In [23]:
# === Predict on test data ===
y_test_pred_log = model_pipeline.predict(X_test)
y_test_pred = np.expm1(y_test_pred_log)

In [26]:
# === Wrap in DataFrame (e.g., for submission) ===
submission = pd.DataFrame({
  "Id": X_test["Id"],  # assuming 'Id' is in test data
  "SalePrice": y_test_pred
})

submission.head()

,Id,SalePrice
0,1461,124202.372111
1,1462,154596.987419
2,1463,179139.570973
3,1464,183541.290397
4,1465,195917.291446


In [ ]:
submission.to_csv("../data/submission.csv", index=False)